In [10]:
# MEA Data Visualization - Corrected Implementation

import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.widgets import RectangleSelector
import seaborn as sns
import h5py
import McsPy
from McsPy import ureg, Q_
from ipywidgets import interact, interactive, fixed, widgets
from IPython.display import display, clear_output


def load_mea_data(file_path):
    channel_raw_data = McsPy.McsData.RawData(file_path)
    analog_stream_0 = channel_raw_data.recordings[0].analog_streams[0]
    stream = analog_stream_0
    keylist = []
    for key in stream.channel_infos.keys():
        keylist.append(key)
        
    channel_id = keylist[0]
    tick = stream.channel_infos[channel_id].info['Tick']
    time = stream.get_channel_sample_timestamps(channel_id)
    first_recording_timepoint = time[0][0]
    scale_factor_for_second = Q_(1,time[1]).to(ureg.s).magnitude
    scale_factor_for_milisecond = scale_factor_for_second/1000
    time_in_sec = time[0]*scale_factor_for_second
    timelengthrecording_ms = time[0][-1]+tick
    timelengthrecording_s = (time[0][-1]+tick)*scale_factor_for_second
    fs = int(stream.channel_infos[channel_id].sampling_frequency.magnitude)
    
    analog_stream_0_data = analog_stream_0.channel_data
    np_analog_stream_0_data = np.transpose(
        channel_raw_data.recordings[0].analog_streams[0].channel_data
        )
    
    # the stream needs to be changed because MCS hates me
    np_analog_for_filter = np.transpose(np_analog_stream_0_data)
    
    return {
        'analog_stream': analog_stream_0,
        'np_analog_for_filter': np_analog_for_filter,
        'time_in_sec': time_in_sec,
        'timelengthrecording_s': timelengthrecording_s,
        'fs': fs,
        'tick': tick,
        'scale_factor_for_second': scale_factor_for_second
    }

def get_mea_signal(data, channel_idx, from_in_s=0, to_in_s=None):
    np_analog_for_filter = data['np_analog_for_filter']
    time_in_sec = data['time_in_sec']
    
    if to_in_s is None:
        to_in_s = time_in_sec[-1]
    
    from_idx = max(0, int(from_in_s * data['fs']))
    to_idx = min(np_analog_for_filter.shape[1], int(to_in_s * data['fs']))
    
    signal = np_analog_for_filter[channel_idx, from_idx:to_idx]
    time = time_in_sec[from_idx:to_idx]
    
    return signal, time

def plot_mea_trace(data, channel_label):
    channel_idx = data['channel_map'][channel_label]
    signal, time = get_mea_signal(data, channel_idx)
    
    plt.figure(figsize=(12, 6))
    plt.plot(time, signal)
    plt.title(f'MEA Trace - Channel {channel_label}')
    plt.xlabel('Time (s)')
    plt.ylabel('Voltage (µV)')
    plt.grid(True)

    def line_select_callback(eclick, erelease):
        x1, y1 = eclick.xdata, eclick.ydata
        x2, y2 = erelease.xdata, erelease.ydata
        plt.xlim(min(x1, x2), max(x1, x2))
        plt.ylim(min(y1, y2), max(y1, y2))
        plt.draw()

    rs = RectangleSelector(plt.gca(), line_select_callback, useblit=True,
                           button=[1, 3],  # Left and right mouse buttons
                           minspanx=5, minspany=5,
                           spancoords='pixels',
                           interactive=True)

    plt.show()

def create_mea_grid(data):
    columns = ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'R']
    rows = range(1, 17)

    def on_button_clicked(b):
        plot_mea_trace(data, b.description)

    grid_layout = widgets.GridspecLayout(17, 17, width='auto', height='auto')

    # Add column labels
    for i, col in enumerate(columns):
        grid_layout[0, i+1] = widgets.Label(value=col, layout=widgets.Layout(width='auto', height='auto'))

    # Add row labels and buttons
    for row in rows:
        grid_layout[row, 0] = widgets.Label(value=str(row), layout=widgets.Layout(width='auto', height='auto'))
        for col in columns:
            channel_label = f"{col}{row}"
            if channel_label in data['channel_map'] and channel_label not in ['A1', 'A16', 'R1', 'R16']:
                button = widgets.Button(description=channel_label, 
                                        layout=widgets.Layout(width='auto', height='auto'),
                                        style={'button_color': 'lightblue'})
                button.on_click(on_button_clicked)
                grid_layout[row, columns.index(col) + 1] = button
            else:
                grid_layout[row, columns.index(col) + 1] = widgets.Label(value='', layout=widgets.Layout(width='auto', height='auto'))

    display(grid_layout)

# Main execution
file_path = input("Enter the path to your .h5 file: ")
data = load_mea_data(file_path)

# Create channel map
columns = ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'R']
rows = range(1, 17)
channel_map = {f"{col}{row}": i for i, (col, row) in enumerate([(col, row) for row in rows for col in columns])}
channel_map = {k: v for k, v in channel_map.items() if k not in ['A1', 'A16', 'R1', 'R16']}
data['channel_map'] = channel_map

print("Data loaded successfully!")
print(f"Number of channels: {data['np_analog_for_filter'].shape[0]}")
print(f"Recording duration: {data['timelengthrecording_s']:.2f} seconds")

# Create MEA grid for channel selection
create_mea_grid(data)

Enter the path to your .h5 file:  /Users/naila/Documents/DATA/_temp_data_to_analyze/220329_mousecortex_div13/2022-03-29T14-11-14__mousecortex_div13_biometra_ID020_100µMCapsaicin_spont_1__.h5


Recording_0 <HDF5 group "/Data/Recording_0" (2 members)>
Stream_0 <HDF5 group "/Data/Recording_0/AnalogStream/Stream_0" (3 members)>
ChannelData <HDF5 dataset "ChannelData": shape (252, 3012000), type "<i4">
ChannelDataTimeStamps <HDF5 dataset "ChannelDataTimeStamps": shape (1, 3), type "<i8">
InfoChannel <HDF5 dataset "InfoChannel": shape (252,), type "|V108">
Data loaded successfully!
Number of channels: 252
Recording duration: 301.20 seconds


GridspecLayout(children=(Label(value='A', layout=Layout(grid_area='widget001', height='auto', width='auto')), …

In [18]:
import os
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.widgets import RectangleSelector
import h5py
import McsPy
from McsPy import ureg, Q_
from ipywidgets import widgets
from IPython.display import display, clear_output


def load_mea_data(file_path):
    channel_raw_data = McsPy.McsData.RawData(file_path)
    analog_stream_0 = channel_raw_data.recordings[0].analog_streams[0]
    stream = analog_stream_0
    keylist = []
    for key in stream.channel_infos.keys():
        keylist.append(key)
        
    channel_id = keylist[0]
    tick = stream.channel_infos[channel_id].info['Tick']
    time = stream.get_channel_sample_timestamps(channel_id)
    first_recording_timepoint = time[0][0]
    scale_factor_for_second = Q_(1, time[1]).to(ureg.s).magnitude
    scale_factor_for_millisecond = scale_factor_for_second / 1000
    time_in_sec = time[0] * scale_factor_for_second
    timelengthrecording_ms = time[0][-1] + tick
    timelengthrecording_s = (time[0][-1] + tick) * scale_factor_for_second
    fs = int(stream.channel_infos[channel_id].sampling_frequency.magnitude)
    
    analog_stream_0_data = analog_stream_0.channel_data
    np_analog_stream_0_data = np.transpose(channel_raw_data.recordings[0].analog_streams[0].channel_data)
    
    np_analog_for_filter = np.transpose(np_analog_stream_0_data)
    
    return {
        'analog_stream': analog_stream_0,
        'np_analog_for_filter': np_analog_for_filter,
        'time_in_sec': time_in_sec,
        'timelengthrecording_s': timelengthrecording_s,
        'fs': fs,
        'tick': tick,
        'scale_factor_for_second': scale_factor_for_second
    }


def get_mea_signal(data, channel_idx, from_in_s=0, to_in_s=None):
    np_analog_for_filter = data['np_analog_for_filter']
    time_in_sec = data['time_in_sec']
    
    if to_in_s is None:
        to_in_s = time_in_sec[-1]
    
    from_idx = max(0, int(from_in_s * data['fs']))
    to_idx = min(np_analog_for_filter.shape[1], int(to_in_s * data['fs']))
    
    signal = np_analog_for_filter[channel_idx, from_idx:to_idx]
    time = time_in_sec[from_idx:to_idx]
    
    return signal, time


def plot_mea_trace(data, channel_label):
    channel_idx = data['channel_map'][channel_label]
    signal, time = get_mea_signal(data, channel_idx)
    
    # Clear previous output
    clear_output(wait=True)

    # Create the plot
    plt.figure(figsize=(12, 6))
    plt.plot(time, signal)
    plt.title(f'MEA Trace - Channel {channel_label}')
    plt.xlabel('Time (s)')
    plt.ylabel('Voltage (µV)')
    plt.grid(True)

    # Add interactive selector for zooming in on a specific part of the trace
    def line_select_callback(eclick, erelease):
        x1, y1 = eclick.xdata, eclick.ydata
        x2, y2 = erelease.xdata, erelease.ydata
        plt.xlim(min(x1, x2), max(x1, x2))
        plt.ylim(min(y1, y2), max(y1, y2))
        plt.draw()

    rs = RectangleSelector(plt.gca(), line_select_callback, useblit=True,
                           button=[1, 3],  # Left and right mouse buttons
                           minspanx=5, minspany=5,
                           spancoords='pixels',
                           interactive=True)

    plt.show()


def create_mea_grid(data):
    columns = ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'R']
    rows = range(1, 17)

    def on_button_clicked(b):
        plot_mea_trace(data, b.description)

    grid_layout = widgets.GridspecLayout(17, 17, width='auto', height='auto')

    # Add column labels
    for i, col in enumerate(columns):
        grid_layout[0, i+1] = widgets.Label(value=col, layout=widgets.Layout(width='auto', height='auto'))

    # Add row labels and buttons
    for row in rows:
        grid_layout[row, 0] = widgets.Label(value=str(row), layout=widgets.Layout(width='auto', height='auto'))
        for col in columns:
            channel_label = f"{col}{row}"
            if channel_label in data['channel_map'] and channel_label not in ['A1', 'A16', 'R1', 'R16']:
                button = widgets.Button(description=channel_label, 
                                        layout=widgets.Layout(width='auto', height='auto'),
                                        style={'button_color': 'lightblue'})
                button.on_click(on_button_clicked)
                grid_layout[row, columns.index(col) + 1] = button
            else:
                grid_layout[row, columns.index(col) + 1] = widgets.Label(value='', layout=widgets.Layout(width='auto', height='auto'))

    display(grid_layout)


# Main execution
file_path = input("Enter the path to your .h5 file: ")
data = load_mea_data(file_path)

# Create channel map
columns = ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'R']
rows = range(1, 17)
channel_map = {f"{col}{row}": i for i, (col, row) in enumerate([(col, row) for row in rows for col in columns])}
channel_map = {k: v for k, v in channel_map.items() if k not in ['A1', 'A16', 'R1', 'R16']}
data['channel_map'] = channel_map

print("Data loaded successfully!")
print(f"Number of channels: {data['np_analog_for_filter'].shape[0]}")
print(f"Recording duration: {data['timelengthrecording_s']:.2f} seconds")

# Create MEA grid for channel selection
create_mea_grid(data)


Enter the path to your .h5 file:  /Users/naila/Documents/DATA/_temp_data_to_analyze/220329_mousecortex_div13/2022-03-29T14-11-14__mousecortex_div13_biometra_ID020_100µMCapsaicin_spont_1__.h5


Recording_0 <HDF5 group "/Data/Recording_0" (2 members)>
Stream_0 <HDF5 group "/Data/Recording_0/AnalogStream/Stream_0" (3 members)>
ChannelData <HDF5 dataset "ChannelData": shape (252, 3012000), type "<i4">
ChannelDataTimeStamps <HDF5 dataset "ChannelDataTimeStamps": shape (1, 3), type "<i8">
InfoChannel <HDF5 dataset "InfoChannel": shape (252,), type "|V108">
Data loaded successfully!
Number of channels: 252
Recording duration: 301.20 seconds


GridspecLayout(children=(Label(value='A', layout=Layout(grid_area='widget001', height='auto', width='auto')), …

In [14]:
import os
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.widgets import RectangleSelector
import h5py
import McsPy
from McsPy import ureg, Q_
from ipywidgets import widgets
from IPython.display import display, clear_output


def load_mea_data(file_path):
    channel_raw_data = McsPy.McsData.RawData(file_path)
    analog_stream_0 = channel_raw_data.recordings[0].analog_streams[0]
    stream = analog_stream_0
    keylist = []
    for key in stream.channel_infos.keys():
        keylist.append(key)
        
    channel_id = keylist[0]
    tick = stream.channel_infos[channel_id].info['Tick']
    time = stream.get_channel_sample_timestamps(channel_id)
    first_recording_timepoint = time[0][0]
    scale_factor_for_second = Q_(1, time[1]).to(ureg.s).magnitude
    scale_factor_for_millisecond = scale_factor_for_second / 1000
    time_in_sec = time[0] * scale_factor_for_second
    timelengthrecording_ms = time[0][-1] + tick
    timelengthrecording_s = (time[0][-1] + tick) * scale_factor_for_second
    fs = int(stream.channel_infos[channel_id].sampling_frequency.magnitude)
    
    analog_stream_0_data = analog_stream_0.channel_data
    np_analog_stream_0_data = np.transpose(channel_raw_data.recordings[0].analog_streams[0].channel_data)
    
    np_analog_for_filter = np.transpose(np_analog_stream_0_data)
    
    return {
        'analog_stream': analog_stream_0,
        'np_analog_for_filter': np_analog_for_filter,
        'time_in_sec': time_in_sec,
        'timelengthrecording_s': timelengthrecording_s,
        'fs': fs,
        'tick': tick,
        'scale_factor_for_second': scale_factor_for_second
    }


def get_mea_signal(data, channel_idx, from_in_s=0, to_in_s=None):
    np_analog_for_filter = data['np_analog_for_filter']
    time_in_sec = data['time_in_sec']
    
    if to_in_s is None:
        to_in_s = time_in_sec[-1]
    
    from_idx = max(0, int(from_in_s * data['fs']))
    to_idx = min(np_analog_for_filter.shape[1], int(to_in_s * data['fs']))
    
    signal = np_analog_for_filter[channel_idx, from_idx:to_idx]
    time = time_in_sec[from_idx:to_idx]
    
    return signal, time


def plot_mea_trace(data, channel_label):
    if channel_label not in data['channel_map']:
        print(f"Channel {channel_label} does not exist.")
        return
    
    channel_idx = data['channel_map'][channel_label]
    signal, time = get_mea_signal(data, channel_idx)
    
    # Clear previous output
    clear_output(wait=True)

    # Create the plot
    plt.figure(figsize=(12, 6))
    plt.plot(time, signal)
    plt.title(f'MEA Trace - Channel {channel_label}')
    plt.xlabel('Time (s)')
    plt.ylabel('Voltage (µV)')
    plt.grid(True)

    # Add interactive selector for zooming in on a specific part of the trace
    def line_select_callback(eclick, erelease):
        x1, y1 = eclick.xdata, eclick.ydata
        x2, y2 = erelease.xdata, erelease.ydata
        plt.xlim(min(x1, x2), max(x1, x2))
        plt.ylim(min(y1, y2), max(y1, y2))
        plt.draw()

    rs = RectangleSelector(plt.gca(), line_select_callback, useblit=True,
                           button=[1, 3],  # Left and right mouse buttons
                           minspanx=5, minspany=5,
                           spancoords='pixels',
                           interactive=True)

    plt.show()


def create_channel_input(data):
    channel_input = widgets.Text(
        description='Channel:',
        placeholder='Enter channel label (e.g., O2)',
        layout=widgets.Layout(width='300px')
    )
    
    def on_submit(change):
        plot_mea_trace(data, channel_input.value)
    
    channel_input.on_submit(on_submit)
    display(channel_input)


# Main execution
file_path = input("Enter the path to your .h5 file: ")
data = load_mea_data(file_path)

# Create channel map
columns = ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'R']
rows = range(1, 17)
channel_map = {f"{col}{row}": i for i, (col, row) in enumerate([(col, row) for row in rows for col in columns])}
channel_map = {k: v for k, v in channel_map.items() if k not in ['A1', 'A16', 'R1', 'R16']}
data['channel_map'] = channel_map

print("Data loaded successfully!")
print(f"Number of channels: {data['np_analog_for_filter'].shape[0]}")
print(f"Recording duration: {data['timelengthrecording_s']:.2f} seconds")

# Create input box for channel selection
create_channel_input(data)


Enter the path to your .h5 file:  /Users/naila/Documents/DATA/_temp_data_to_analyze/220329_mousecortex_div13/2022-03-29T14-11-14__mousecortex_div13_biometra_ID020_100µMCapsaicin_spont_1__.h5


Recording_0 <HDF5 group "/Data/Recording_0" (2 members)>
Stream_0 <HDF5 group "/Data/Recording_0/AnalogStream/Stream_0" (3 members)>
ChannelData <HDF5 dataset "ChannelData": shape (252, 3012000), type "<i4">
ChannelDataTimeStamps <HDF5 dataset "ChannelDataTimeStamps": shape (1, 3), type "<i8">
InfoChannel <HDF5 dataset "InfoChannel": shape (252,), type "|V108">
Data loaded successfully!
Number of channels: 252
Recording duration: 301.20 seconds


/var/folders/mw/vcc1cc753tb6d1bcgk2hwhpr0000gn/T/ipykernel_62606/1658827965.py:109: DeprecationWarning: on_submit is deprecated. Instead, set the .continuous_update attribute to False and observe the value changing with: mywidget.observe(callback, 'value').
  channel_input.on_submit(on_submit)


Text(value='', description='Channel:', layout=Layout(width='300px'), placeholder='Enter channel label (e.g., O…

In [20]:
import os
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.widgets import RectangleSelector
import h5py
import McsPy
from McsPy import ureg, Q_
from ipywidgets import widgets
from IPython.display import display, clear_output


def load_mea_data(file_path):
    channel_raw_data = McsPy.McsData.RawData(file_path)
    analog_stream_0 = channel_raw_data.recordings[0].analog_streams[0]
    stream = analog_stream_0
    keylist = []
    for key in stream.channel_infos.keys():
        keylist.append(key)
        
    channel_id = keylist[0]
    tick = stream.channel_infos[channel_id].info['Tick']
    time = stream.get_channel_sample_timestamps(channel_id)
    first_recording_timepoint = time[0][0]
    scale_factor_for_second = Q_(1, time[1]).to(ureg.s).magnitude
    scale_factor_for_millisecond = scale_factor_for_second / 1000
    time_in_sec = time[0] * scale_factor_for_second
    timelengthrecording_ms = time[0][-1] + tick
    timelengthrecording_s = (time[0][-1] + tick) * scale_factor_for_second
    fs = int(stream.channel_infos[channel_id].sampling_frequency.magnitude)
    
    analog_stream_0_data = analog_stream_0.channel_data
    np_analog_stream_0_data = np.transpose(channel_raw_data.recordings[0].analog_streams[0].channel_data)
    
    np_analog_for_filter = np.transpose(np_analog_stream_0_data)
    
    return {
        'analog_stream': analog_stream_0,
        'np_analog_for_filter': np_analog_for_filter,
        'time_in_sec': time_in_sec,
        'timelengthrecording_s': timelengthrecording_s,
        'fs': fs,
        'tick': tick,
        'scale_factor_for_second': scale_factor_for_second
    }


def get_mea_signal(data, channel_idx, from_in_s=0, to_in_s=None):
    np_analog_for_filter = data['np_analog_for_filter']
    time_in_sec = data['time_in_sec']
    
    if to_in_s is None:
        to_in_s = time_in_sec[-1]
    
    from_idx = max(0, int(from_in_s * data['fs']))
    to_idx = min(np_analog_for_filter.shape[1], int(to_in_s * data['fs']))
    
    signal = np_analog_for_filter[channel_idx, from_idx:to_idx]
    time = time_in_sec[from_idx:to_idx]
    
    return signal, time


def plot_mea_trace(data, channel_label):
    if channel_label not in data['channel_map']:
        print(f"Channel {channel_label} does not exist.")
        return
    
    channel_idx = data['channel_map'][channel_label]
    signal, time = get_mea_signal(data, channel_idx)
    
    # Clear previous output
    clear_output(wait=True)

    # Create the plot
    plt.figure(figsize=(12, 6))
    plt.plot(time, signal)
    plt.title(f'MEA Trace - Channel {channel_label}')
    plt.xlabel('Time (s)')
    plt.ylabel('Voltage (µV)')
    plt.grid(True)

    # Add interactive selector for zooming in on a specific part of the trace
    def line_select_callback(eclick, erelease):
        x1, y1 = eclick.xdata, eclick.ydata
        x2, y2 = erelease.xdata, y2
        plt.xlim(min(x1, x2), max(x1, x2))
        plt.ylim(min(y1, y2), max(y1, y2))
        plt.draw()

    rs = RectangleSelector(plt.gca(), line_select_callback, useblit=True,
                           button=[1, 3],  # Left and right mouse buttons
                           minspanx=5, minspany=5,
                           spancoords='pixels',
                           interactive=True)

    plt.show()


def create_channel_input(data):
    channel_input = widgets.Text(
        description='Channel:',
        placeholder='Enter channel label (e.g., O2)',
        layout=widgets.Layout(width='300px')
    )
    
    def on_channel_change(change):
        # Only act on changes when the user presses Enter
        if change['type'] == 'change' and change['name'] == 'value':
            plot_mea_trace(data, channel_input.value)

    channel_input.observe(on_channel_change, names='value')
    display(channel_input)


# Main execution
file_path = input("Enter the path to your .h5 file: ")
data = load_mea_data(file_path)

# Create channel map
columns = ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'R']
rows = range(1, 17)
channel_map = {f"{col}{row}": i for i, (col, row) in enumerate([(col, row) for row in rows for col in columns])}
channel_map = {k: v for k, v in channel_map.items() if k not in ['A1', 'A16', 'R1', 'R16']}
data['channel_map'] = channel_map

print("Data loaded successfully!")
print(f"Number of channels: {data['np_analog_for_filter'].shape[0]}")
print(f"Recording duration: {data['timelengthrecording_s']:.2f} seconds")

# Create input box for channel selection
create_channel_input(data)


Enter the path to your .h5 file:  /Users/naila/Documents/DATA/_temp_data_to_analyze/220329_mousecortex_div13/2022-03-29T14-11-14__mousecortex_div13_biometra_ID020_100µMCapsaicin_spont_1__.h5


Recording_0 <HDF5 group "/Data/Recording_0" (2 members)>
Stream_0 <HDF5 group "/Data/Recording_0/AnalogStream/Stream_0" (3 members)>
ChannelData <HDF5 dataset "ChannelData": shape (252, 3012000), type "<i4">
ChannelDataTimeStamps <HDF5 dataset "ChannelDataTimeStamps": shape (1, 3), type "<i8">
InfoChannel <HDF5 dataset "InfoChannel": shape (252,), type "|V108">
Data loaded successfully!
Number of channels: 252
Recording duration: 301.20 seconds


Text(value='', description='Channel:', layout=Layout(width='300px'), placeholder='Enter channel label (e.g., O…

In [26]:
import os
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.widgets import RectangleSelector
import h5py
import McsPy
from McsPy import ureg, Q_
from ipywidgets import widgets
from IPython.display import display, clear_output


def load_mea_data(file_path):
    channel_raw_data = McsPy.McsData.RawData(file_path)
    analog_stream_0 = channel_raw_data.recordings[0].analog_streams[0]
    stream = analog_stream_0
    keylist = []
    for key in stream.channel_infos.keys():
        keylist.append(key)
        
    channel_id = keylist[0]
    tick = stream.channel_infos[channel_id].info['Tick']
    time = stream.get_channel_sample_timestamps(channel_id)
    first_recording_timepoint = time[0][0]
    scale_factor_for_second = Q_(1, time[1]).to(ureg.s).magnitude
    scale_factor_for_millisecond = scale_factor_for_second / 1000
    time_in_sec = time[0] * scale_factor_for_second
    timelengthrecording_ms = time[0][-1] + tick
    timelengthrecording_s = (time[0][-1] + tick) * scale_factor_for_second
    fs = int(stream.channel_infos[channel_id].sampling_frequency.magnitude)
    
    analog_stream_0_data = analog_stream_0.channel_data
    np_analog_stream_0_data = np.transpose(channel_raw_data.recordings[0].analog_streams[0].channel_data)
    
    np_analog_for_filter = np.transpose(np_analog_stream_0_data)
    
    return {
        'analog_stream': analog_stream_0,
        'np_analog_for_filter': np_analog_for_filter,
        'time_in_sec': time_in_sec,
        'timelengthrecording_s': timelengthrecording_s,
        'fs': fs,
        'tick': tick,
        'scale_factor_for_second': scale_factor_for_second
    }


def get_mea_signal(data, channel_idx, from_in_s=0, to_in_s=None):
    np_analog_for_filter = data['np_analog_for_filter']
    time_in_sec = data['time_in_sec']
    
    if to_in_s is None:
        to_in_s = time_in_sec[-1]
    
    from_idx = max(0, int(from_in_s * data['fs']))
    to_idx = min(np_analog_for_filter.shape[1], int(to_in_s * data['fs']))
    
    signal = np_analog_for_filter[channel_idx, from_idx:to_idx]
    time = time_in_sec[from_idx:to_idx]
    
    return signal, time


def plot_mea_trace(data, channel_label):
    if channel_label not in data['channel_map']:
        print(f"Channel {channel_label} does not exist.")
        return
    
    channel_idx = data['channel_map'][channel_label]
    signal, time = get_mea_signal(data, channel_idx)
    
    # Clear previous output
    clear_output(wait=True)

    # Create the plot
    plt.figure(figsize=(12, 6))
    plt.plot(time, signal)
    plt.title(f'MEA Trace - Channel {channel_label}')
    plt.xlabel('Time (s)')
    plt.ylabel('Voltage (µV)')
    plt.grid(True)

    # Add interactive selector for zooming in on a specific part of the trace
    def line_select_callback(eclick, erelease):
        x1, y1 = eclick.xdata, eclick.ydata
        x2, y2 = erelease.xdata, erelease.ydata
        plt.xlim(min(x1, x2), max(x1, x2))
        plt.ylim(min(y1, y2), max(y1, y2))
        plt.draw()

    rs = RectangleSelector(plt.gca(), line_select_callback, useblit=True,
                           button=[1, 3],  # Left and right mouse buttons
                           minspanx=5, minspany=5,
                           spancoords='pixels',
                           interactive=True)

    plt.show()


def create_channel_input(data):
    channel_input = widgets.Text(
        description='Channel:',
        placeholder='Enter channel label (e.g., O2)',
        layout=widgets.Layout(width='300px')
    )
    
    output = widgets.Output()

    def on_submit(change):
        with output:
            clear_output(wait=True)
            plot_mea_trace(data, channel_input.value)

    # Trigger the function when Enter is pressed
    channel_input.on_submit(on_submit)
    
    # Display the input and output widget
    display(channel_input, output)


# Main execution
file_path = input("Enter the path to your .h5 file: ")
data = load_mea_data(file_path)

# Create channel map
columns = ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'R']
rows = range(1, 17)
channel_map = {f"{col}{row}": i for i, (col, row) in enumerate([(col, row) for row in rows for col in columns])}
channel_map = {k: v for k, v in channel_map.items() if k not in ['A1', 'A16', 'R1', 'R16']}
data['channel_map'] = channel_map

print("Data loaded successfully!")
print(f"Number of channels: {data['np_analog_for_filter'].shape[0]}")
print(f"Recording duration: {data['timelengthrecording_s']:.2f} seconds")

# Create input box for channel selection
create_channel_input(data)


Enter the path to your .h5 file:  /Users/naila/Documents/DATA/_temp_data_to_analyze/trial_new_script/2022-03-29T09-30-05__mousecortex_div13_biometra_ID018_nodrug_spont_1__.h5


Recording_0 <HDF5 group "/Data/Recording_0" (2 members)>
Stream_0 <HDF5 group "/Data/Recording_0/AnalogStream/Stream_0" (3 members)>
ChannelData <HDF5 dataset "ChannelData": shape (252, 3258000), type "<i4">
ChannelDataTimeStamps <HDF5 dataset "ChannelDataTimeStamps": shape (1, 3), type "<i8">
InfoChannel <HDF5 dataset "InfoChannel": shape (252,), type "|V108">
Data loaded successfully!
Number of channels: 252
Recording duration: 333.00 seconds


/var/folders/mw/vcc1cc753tb6d1bcgk2hwhpr0000gn/T/ipykernel_62606/1640561644.py:114: DeprecationWarning: on_submit is deprecated. Instead, set the .continuous_update attribute to False and observe the value changing with: mywidget.observe(callback, 'value').
  channel_input.on_submit(on_submit)


Text(value='', description='Channel:', layout=Layout(width='300px'), placeholder='Enter channel label (e.g., O…

Output()

In [36]:
import os
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.widgets import RectangleSelector
import h5py
import McsPy
from McsPy import ureg, Q_
from ipywidgets import widgets
from IPython.display import display, clear_output

# Enable interactive mode for Jupyter
# Enable interactive mode for Jupyter
%matplotlib widget



def load_mea_data(file_path):
    channel_raw_data = McsPy.McsData.RawData(file_path)
    analog_stream_0 = channel_raw_data.recordings[0].analog_streams[0]
    stream = analog_stream_0
    keylist = []
    for key in stream.channel_infos.keys():
        keylist.append(key)
        
    channel_id = keylist[0]
    tick = stream.channel_infos[channel_id].info['Tick']
    time = stream.get_channel_sample_timestamps(channel_id)
    first_recording_timepoint = time[0][0]
    scale_factor_for_second = Q_(1, time[1]).to(ureg.s).magnitude
    scale_factor_for_millisecond = scale_factor_for_second / 1000
    time_in_sec = time[0] * scale_factor_for_second
    timelengthrecording_ms = time[0][-1] + tick
    timelengthrecording_s = (time[0][-1] + tick) * scale_factor_for_second
    fs = int(stream.channel_infos[channel_id].sampling_frequency.magnitude)
    
    analog_stream_0_data = analog_stream_0.channel_data
    np_analog_stream_0_data = np.transpose(channel_raw_data.recordings[0].analog_streams[0].channel_data)
    
    np_analog_for_filter = np.transpose(np_analog_stream_0_data)
    
    return {
        'analog_stream': analog_stream_0,
        'np_analog_for_filter': np_analog_for_filter,
        'time_in_sec': time_in_sec,
        'timelengthrecording_s': timelengthrecording_s,
        'fs': fs,
        'tick': tick,
        'scale_factor_for_second': scale_factor_for_second
    }


def get_mea_signal(data, channel_idx, from_in_s=0, to_in_s=None):
    np_analog_for_filter = data['np_analog_for_filter']
    time_in_sec = data['time_in_sec']
    
    if to_in_s is None:
        to_in_s = time_in_sec[-1]
    
    from_idx = max(0, int(from_in_s * data['fs']))
    to_idx = min(np_analog_for_filter.shape[1], int(to_in_s * data['fs']))
    
    signal = np_analog_for_filter[channel_idx, from_idx:to_idx]
    time = time_in_sec[from_idx:to_idx]
    
    return signal, time


def plot_mea_trace(data, channel_label):
    if channel_label not in data['channel_map']:
        print(f"Channel {channel_label} does not exist.")
        return
    
    channel_idx = data['channel_map'][channel_label]
    signal, time = get_mea_signal(data, channel_idx)
    
    # Clear previous output
    clear_output(wait=True)

    # Create the plot
    fig, ax = plt.subplots(figsize=(12, 6))
    ax.plot(time, signal)
    ax.set_title(f'MEA Trace - Channel {channel_label}')
    ax.set_xlabel('Time (s)')
    ax.set_ylabel('Voltage (µV)')
    ax.grid(True)

    # Add interactive selector for zooming in on a specific part of the trace
    def line_select_callback(eclick, erelease):
        x1, y1 = eclick.xdata, eclick.ydata
        x2, y2 = erelease.xdata, erelease.ydata
        ax.set_xlim(min(x1, x2), max(x1, x2))
        ax.set_ylim(min(y1, y2), max(y1, y2))
        fig.canvas.draw()

    rs = RectangleSelector(ax, line_select_callback, useblit=True,
                           button=[1, 3],  # Left and right mouse buttons
                           minspanx=5, minspany=5,
                           spancoords='data',
                           interactive=True)

    plt.show()


def create_channel_input(data):
    channel_input = widgets.Text(
        description='Channel:',
        placeholder='Enter channel label (e.g., O2)',
        layout=widgets.Layout(width='300px')
    )
    
    output = widgets.Output()

    def on_submit(change):
        with output:
            clear_output(wait=True)
            plot_mea_trace(data, channel_input.value)

    # Trigger the function when Enter is pressed
    channel_input.on_submit(on_submit)
    
    # Display the input and output widget
    display(channel_input, output)


# Main execution
file_path = input("Enter the path to your .h5 file: ")
data = load_mea_data(file_path)

# Create channel map
columns = ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'R']
rows = range(1, 17)
channel_map = {f"{col}{row}": i for i, (col, row) in enumerate([(col, row) for row in rows for col in columns])}
channel_map = {k: v for k, v in channel_map.items() if k not in ['A1', 'A16', 'R1', 'R16']}
data['channel_map'] = channel_map

print("Data loaded successfully!")
print(f"Number of channels: {data['np_analog_for_filter'].shape[0]}")
print(f"Recording duration: {data['timelengthrecording_s']:.2f} seconds")

# Create input box for channel selection
create_channel_input(data)


RuntimeError: 'widget is not a recognised GUI loop or backend name